In [87]:
RATE = 0.001
NUM_EPOCHS = 11
VALSPLIT = 0.0
np.random.seed(5)
TRAIN_BATCH_SIZE = 2024

In [88]:
X = pd.read_pickle("input/X_HB.pkl")[:]
X.drop(['PU','pt'],1,inplace=True)
mu,std = np.mean(X, axis=0), np.std(X, axis=0)
X = (X-mu)/std
Y = pd.read_pickle("input/Y_HB.pkl")[:][['genE']]

In [69]:
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, BatchNormalization, Input, Dropout, Activation, concatenate, GRU, Dropout
import keras.backend as K
from keras.optimizers import Adam, Nadam, SGD

In [70]:
inputs_keras = Input(shape=(X.shape[1],), name='input')
h = Dropout(rate=RATE)(inputs_keras)
h = Dense(36, activation='relu')(h)
norm = Dropout(rate=RATE)(h)
h = Dense(11, activation = 'relu')(norm)
norm = Dropout(rate=RATE)(h)
h = Dense(3, activation = 'relu')(norm)
net_keras = Dense(1, activation='linear', name='output')(h)

In [71]:
model_keras = Model(inputs=inputs_keras, outputs=net_keras)
model_keras.compile(optimizer=Adam(), loss='mse')

In [77]:

#model_keras.fit(X.values, Y.values,batch_size=2024, epochs=NUM_EPOCHS, shuffle=True,validation_data=(X, Y))
idx = np.random.permutation(X.shape[0])
train_idx = idx[int(VALSPLIT*len(idx)):]
val_idx = idx[:int(VALSPLIT*len(idx))]
# model_keras.fit(X.values[train_idx], Y.values[train_idx],batch_size=2024, 
#                 epochs=NUM_EPOCHS, shuffle=True, validation_data = (X.values[val_idx], Y.values[val_idx]))

model_keras.fit(X.values[train_idx], Y.values[train_idx],batch_size=TRAIN_BATCH_SIZE, 
                epochs=NUM_EPOCHS, shuffle=True)#, validation_data = (X.values[val_idx], Y.values[val_idx]))

result = model_keras.predict(X, 10000)
print(result[:10])

Epoch 1/11
574212/574212 [==============================] - 2s 3us/step - loss: 0.5613
Epoch 2/11
574212/574212 [==============================] - 2s 3us/step - loss: 0.6479
Epoch 3/11
574212/574212 [==============================] - 2s 3us/step - loss: 0.6001
Epoch 4/11
574212/574212 [==============================] - 2s 3us/step - loss: 0.5338
Epoch 5/11
574212/574212 [==============================] - 2s 3us/step - loss: 0.5457
Epoch 6/11
574212/574212 [==============================] - 2s 3us/step - loss: 0.5593
Epoch 7/11
574212/574212 [==============================] - 2s 3us/step - loss: 0.5075
Epoch 8/11
574212/574212 [==============================] - 2s 3us/step - loss: 0.5462
Epoch 9/11
574212/574212 [==============================] - 2s 3us/step - loss: 0.5233
Epoch 10/11
574212/574212 [==============================] - 2s 3us/step - loss: 0.5164
Epoch 11/11
574212/574212 [==============================] - 2s 3us/step - loss: 0.5758
[[ 0.2982934 ]
 [ 1.1861517 ]
 [31.529543

In [187]:
import os, sys, time, threading
import tensorflow as tf
import numpy as np
import pandas as pd
# Disable depreciation warnings and limit verbosity during training
from tensorflow.python.util import deprecation

In [188]:
MODEL_DIR = 'gs://harrisgroup-ctpu/facile/model/'
DATA_DIR = 'gs://harrisgroup-ctpu/facile/data/'
TPU_NAME='uw-tpu'
ZONE_NAME='us-central1-b'
PROJECT_NAME = 'harrisgroup-223921'
NUM_ITERATIONS = 100000 # Number of iterations per TPU training loop
TRAIN_STEPS = 70000
NUM_SHARDS = 8 # Number of shards (TPU chips).
USE_TPU = True

lock = threading.Lock()

#tf.enable_eager_execution()

# DEFINE THE NETWORK

class ModelFacile(object):
    def __call__(self, inputs):
        net = tf.layers.dropout(inputs, rate = 0.001)
        net = tf.layers.dense(net, 36, activation = 'relu')
        norm = tf.layers.dropout(net, rate = 0.001)
        net = tf.layers.dense(norm, 11, activation = 'relu')
        norm = tf.layers.dropout(net, rate = 0.001)
        net = tf.layers.dense(norm, 3, activation = 'relu')
        return tf.layers.dense(net, 1, activation = 'linear', name='output')


def input_facile(train_x,train_y,batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((tf.cast(train_x,tf.float32),tf.cast(train_y,tf.float32)))
    return dataset.apply(tf.contrib.data.batch_and_drop_remainder(batch_size))

def model_fn(features, labels, mode, params):
    model = ModelFacile()

    if mode == tf.estimator.ModeKeys.PREDICT:
        predicted_values = model(features)
        #print("Shape of predicted values is ", tf.shape(predicted_values))
        predictions = {
            'probabilities': predicted_values,
        }
        return tf.contrib.tpu.TPUEstimatorSpec(mode, predictions=predictions)

    predicted_values = model(features)
    loss = tf.losses.mean_squared_error(labels=labels,predictions=predicted_values)

    if mode == tf.estimator.ModeKeys.TRAIN:
      optimizer = tf.train.AdagradOptimizer(learning_rate = 0.1)
      if USE_TPU:
        optimizer = tf.contrib.tpu.CrossShardOptimizer(optimizer)
      train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
      return tf.contrib.tpu.TPUEstimatorSpec(mode, loss=loss, train_op=train_op)


# CREATE AND PREDICT WITH TPUS

def create_estimator(batch_size):
    print("Creating the estimator")
    tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(
        TPU_NAME,
        zone=ZONE_NAME,
        project=PROJECT_NAME)

    run_config = tf.contrib.tpu.RunConfig(
        cluster=tpu_cluster_resolver,
        model_dir=MODEL_DIR,
        session_config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True),
        tpu_config=tf.contrib.tpu.TPUConfig(NUM_ITERATIONS, NUM_SHARDS))

    estimator = tf.contrib.tpu.TPUEstimator(
        model_fn=model_fn,
        use_tpu=USE_TPU,
        train_batch_size=batch_size,
        eval_batch_size=batch_size,
        predict_batch_size=batch_size,
        params={"data_dir": DATA_DIR},
        config=run_config)
    return estimator

def predict(data, batch_size,estimator, results=None, times=None, job_id=None):
    assert (results is None and times is None and job_id is None) or not (results is None or times is None or job_id is None)
    lock.acquire()
    start_time = time.time()

    print("Predicting")
    def predict_input_fn(params):
        batch_size = params["batch_size"]
        dataset_predict = tf.data.Dataset.from_tensor_slices(data.astype('float32'))
        return dataset_predict.batch(batch_size)

    predictions = estimator.predict(predict_input_fn)
    
    num_print = 10
    num = 0
    predicted_values = []
    for pred_dict in predictions:
        #print(pred_dict.keys())
        #print(pred_dict.values())
        predicted_values.append(list(pred_dict.values())[0][0])
        num += 1
#         if (num >= num_print):
#             break
    #predicted_values =np.array(predicted_values)/np.sum(predicted_values)
    #print("Number of prediction is ",i)
    print(predicted_values[:10])

    predict_time = time.time() - start_time
    lock.release()

    return predicted_values[:10]

In [189]:
estimator = create_estimator(8)
estimator.train(input_fn=lambda params: input_facile(X,Y,params["batch_size"]),max_steps=TRAIN_STEPS)

Creating the estimator
INFO:tensorflow:Using config: {'_model_dir': 'gs://harrisgroup-ctpu/facile/model/', '_num_ps_replicas': 0, '_save_summary_steps': 100, '_evaluation_master': 'grpc://10.240.1.10:8470', '_cluster': <tensorflow.python.distribute.cluster_resolver.tpu_cluster_resolver.TPUClusterResolver object at 0x7f8bc1c9a4e0>, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8bb84e1a58>, '_global_id_in_cluster': 0, '_tpu_config': TPUConfig(iterations_per_loop=100000, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=2, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None), '_eval_distribute': None, '_save_checkpoints_steps': None, '_experimental_distribute': None, '_is_chief': True, '_log_step_count_steps': None, '_tf_random_seed': None, '_protocol': None, '_master': 'grpc://10.240.1.10:8470', '_train_distribute': None, '_device_fn': None, '_task_type': 'worker', '_service': None, '_keep_checkpoint_every_n

In [190]:
start_time = time.time()
predicted_values = predict(X[:1000],10000,estimator)
predict_time = time.time() - start_time
print("Time used in the prediction is ", predict_time)

Predicting
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:TPU job name worker
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from gs://harrisgroup-ctpu/facile/model/model.ckpt-70000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Init TPU system
INFO:tensorflow:Initialized TPU in 11 seconds
INFO:tensorflow:Starting infeed thread controller.
INFO:tensorflow:Starting outfeed thread controller.
INFO:tensorflow:Initialized dataset iterators in 0 seconds
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of

INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of dat

INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Stop infeed thread controller
INFO:tensorflow:Shutting down InfeedController thread.
INFO:tensorflow:InfeedController received shutdown signal, stopping.
INFO:tensorflow:Infeed thread finished, shutting down.
INFO:tensorflow:infeed marked as finished
INFO:tensorflow:Stop output thread controller
INFO:tensorflow:Shutting down OutfeedController thread.
INFO:tensorflow:OutfeedController received shutdown signal, stopping.
INFO:tensorflow:Outfeed thread finished, shutting down.
INFO:tensorflow:outfeed marked as finished
INFO:tensorflow:Shutdown TPU system.
INFO:tensorflow:prediction_loop marked as finished
INFO:tensorflow:prediction_loop marked as finished
[0.2177884, 1.030732, 32.034073, 0.9037533, 4.6592135, 4.793602, 0.3008669, 22.211794, 3.1745791, 0.22245517]
Time used in the prediction is  14.679689407348633


In [191]:
predicted_values

[0.2177884,
 1.030732,
 32.034073,
 0.9037533,
 4.6592135,
 4.793602,
 0.3008669,
 22.211794,
 3.1745791,
 0.22245517]

1. TRAIN_STEPS*TRAIN_BATCH_SIZE < len(X)
2. Wrong learning rate kills everything

In [192]:
np.set_printoptions(precision=3)
print(np.array(Y["genE"][:10]))

[ 0.415  1.178 31.989  0.206  4.094  4.966  0.356 21.954  3.917  0.178]


In [193]:
print(result[:10])

[[ 0.298]
 [ 1.186]
 [31.53 ]
 [ 1.16 ]
 [ 4.226]
 [ 4.505]
 [ 0.295]
 [21.794]
 [ 3.167]
 [ 0.271]]
